<a href="https://colab.research.google.com/github/m-pearce070/Datasets-4106/blob/main/CSI4106_A1_300199565.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**ASSIGNMENT 1 - EMPIRICAL STUDY OF KNAPSACK PROBLEM**

**1. Group Description**

Group Number: \\
Member Names: Matthew Pearce \\
Member Student Numbers: 200199565 \\

**2. Knapsack Problem**

Give a description of the problem tackled.

**3. Dataset**

Give a description of the dataset used with references.  

**Import important libraries**

In [39]:
import pandas as pd
import itertools
import numpy as np

**Read Dataset**

As outlined in the project description, it should be possible for the correctors to execute your notebook without requiring any downloads.

To facilitate access to the dataset without the need for downloads, you can upload it to a public GitHub repository and provide a link to the raw version of the dataset.

The link to the raw version is as follows:
*https://raw.githubusercontent.com/GITHUB_USERNAME/REPOSITORY_NAME/main/DATASETNAME.csv*

For example:

https://raw.githubusercontent.com/baharin/KnapsackProblem/main/knapsack_5_items.csv

Now provide the link to YOUR dataset and read the dataset using pandas:

In [40]:
url = "https://raw.githubusercontent.com/m-pearce070/Datasets-4106/main/knapsack_5_items.csv"

dataset = pd.read_csv(url)

Let's see what are the columns of the dataset? :

In [41]:
dataset.columns

Index(['Weights', 'Prices', 'Capacity', 'Best picks', 'Best price'], dtype='object')

As we expected, we have columns for weights, costs, capacity, best picks and best price for all the instances.

Now let's see the first 10 entries (rows):

In [42]:
dataset.head(10)

,Weights,Prices,Capacity,Best picks,Best price
0,[46 40 42 38 10],[12 19 19 15 8],40,[0. 1. 0. 0. 0.],19
1,[11 31 4 6 7],[ 2 8 18 16 3],64,[1. 1. 1. 1. 1.],47
2,[32 49 27 37 24],[19 16 16 4 1],87,[1. 0. 1. 0. 1.],36
3,[20 35 22 23 16],[19 17 19 9 1],21,[1. 0. 0. 0. 0.],19
4,[ 7 12 19 13 20],[10 11 18 15 5],50,[0. 1. 1. 1. 0.],44
5,[27 10 25 25 7],[13 19 7 16 3],66,[1. 1. 0. 1. 0.],48
6,[21 2 33 45 26],[ 1 14 10 6 13],80,[0. 1. 1. 0. 1.],37
7,[37 27 39 14 25],[18 7 15 4 13],35,[0. 0. 0. 0. 1.],13
8,[ 1 48 4 23 39],[ 9 4 10 16 12],51,[1. 0. 1. 1. 0.],35
9,[ 4 3 22 9 32],[14 6 3 17 8],53,[1. 1. 0. 1. 1.],45


**Preprocessing Step**

Typically, the initial step in any project that involves reading and handling data is data preprocessing and cleansing.

In our dataset, we expect the entries in the "Weights," "Prices," and "Best Picks" columns to be in the form of arrays of floats or integers, like this: [45, 40, 42, 38, 10]

However, when you read each entry using pandas, they will be in a form of a string: "[45 40 42 38 10]"

So we need to convert these strings into "arrays of floats or integers." You can utilize the function provided below for this purpose:


In [43]:
def string_to_list(string):

  string_list = string.strip('[]').split()

  float_list = [float(element) for element in string_list]

  return float_list

Furthermore, it's possible that certain rows in the dataset contain empty values in specific columns. We also aim to eliminate these rows as they do not provide any useful information. We use dropna() function to do so:

In [44]:
#Ignore the warning messages.

dataset = dataset.dropna()

dataset.Weights = dataset.Weights.apply(lambda x : string_to_list(x))
dataset.Prices = dataset.Prices.apply(lambda x : string_to_list(x))
dataset['Best picks'] = dataset['Best picks'].apply(lambda x : string_to_list(x))

Now it's time to implement the search algorithms. For each algorithm, a template is provided to you. You can modify this template if you want. But first you should try to go look at all the parameters used, as they are all important. You can also define any number of auxiliary functions you want.


**4. Generate and Test**

We will attempt to determine the optimal solution using the generate and test method. First, we will generate all permutations of selected items. Then will reduce the selections with respect to the maximum weight (capacity). A selection is valid if the sum of selected item weights is at most capacity. From these valid selections, we will determine which selection produces the highest total price.

This method should find an optimal solution, however it is limited in terms of practical applications. Generating every value within a domain is not always possible and may take longer than other approches.


In [62]:
#global variable to hold permutation table
generatedValues = [[]]

#Return a 2d array containing all binary permutations of length len
#e.g. binary_permutation(2) = [[0, 0], [0, 1], [1, 0], [1, 1]]
def binary_permutation(len):
  numRows = pow(2, len)
  arr = [[0 for i in range(len)] for j in range(numRows)]
  period = 1
  write = False #write head moves up or down depending on period length of column
  counter = 0 #track period

  for col in range(len-1, -1, -1):
    counter = 0
    write = False
    for row in range(numRows):
      if counter == period: #toggle write
        write = not write
        counter = 0
      if write: #write head is down, update values for next period
        arr[row][col] = 1
      counter = counter + 1
    period = period*2 #period is doubled between columns from right to left
  return arr

#get or generate new permutation table
def getGeneratedValues(num):
  global generatedValues
  if num != len(generatedValues[0]): # generate new values if length has changed
    generatedValues = binary_permutation(num)
  return generatedValues

# Weight Function
# Return True iff items selected are within capacity
def validWeight(weights, picks, capacity):
  weight = 0

  for i in range(len(weights)):
    #print("weights, picks", weights,picks)
    weight = weight + (weights[i]*picks[i])
  return weight <= capacity

# Cost Function
# Return total price of provided picks given their price
def getPrice(prices, picks):
  cost = 0
  for i in range(len(prices)):
    cost = cost + (prices[i]*picks[i])
  return cost

#try all solutions and return an optimal solution
def gen_and_test(data):
  arrayLength = len(data[0])
  bestPicks = []
  bestPrice = 0
  currentWeight = 0
  currentPrice = 0

  generatedValues = getGeneratedValues(arrayLength) #get generated values

  for currentPicks in generatedValues:
    currentWeight = 0
    for x in range(arrayLength):
      if validWeight(data[0], currentPicks, data[2]):
        currentPrice = getPrice(data[1], currentPicks)
        if currentPrice > bestPrice: # Keep track of current best solution
          bestPrice = currentPrice
          bestPicks = currentPicks

  return bestPrice, bestPicks

In [46]:
solutions = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = gen_and_test(row)
    solutions.append(1 if target == solution else 0)


In [47]:
# Accuracy
print('Accuracy of best prices found is', np.mean(solutions))

Accuracy of best prices found is 1.0


**Your Analysis:**

By iterating through every possible selection, we can be confident that our best pick is an optimal solution. The solutions we discovered for best pick are precisely the best picks given by the dataset, thus we can be confident about the reliability of the choices provided by the Kaggle dataset.

While we found an optimal solution for each case, it is worth noting that the generate and test method makes no use of heuristics, and therefore is more resource intensive than other approaches.


------------------------------------------------------------------------------------------------

**5. Greedy Search**

Next we will implement a greedy search algorithm. We will select the items with the highest cost first. If two items have the same cost, we will select the item with the lower weight. This process will repeat until no more items can be added without overflowing the capacity of the bag.

In [91]:
# Note: This solution uses the following functions as defined in the generate and test section:
# getPrice(prices, picks)
# validWeight(weights, picks, capacity)

# return a copy of array with array[itemNumber]=num
def updateList(array, itemNumber, num=1):
  newArray = array.copy()
  newArray[itemNumber] = num
  return newArray

# Find repeated prices and return a list of indices for each tie ranked by prices and weights
def tieBreaker(weights, prices):
  pricesSet = set(prices)
  tieData = []

  if len(pricesSet) == len(prices): #there are no two items with the same price
    return [], tieData

  # find which values are repeated
  duplicatedPrices = prices.copy()
  for item in pricesSet:
    duplicatedPrices.remove(item)
  duplicatedPricesSet = set(duplicatedPrices)

  # find the indices of repeated prices
  for item in duplicatedPricesSet:
    indicesArray = []
    for i in range(len(prices)):
      if prices[i] == item:
        indicesArray.append(i)
    tieData.append([item, indicesArray])

  # sort indices for each tie by weight
  for item in tieData:
    for i in range(len(item[1])):
      item[1].sort(key=lambda i: weights[i])

  return duplicatedPricesSet, tieData

# return a list of indices sorted first by price then by weight
def getRanking(weights, prices):
  sortedValues = sorted(prices, reverse=True) # sort values by price
  rankedIndices = [] # list to hold ranked indices
  duplicatedPrices, tieData = tieBreaker(weights, prices) # find ties
  solvedTies = set()

  for value in sortedValues: # rank all indices
    if value not in solvedTies: # if tie has been solved do not continue
      if value in duplicatedPrices: # tie case
        solvedTies.add(value)
        for item in tieData: # find tie data
          if item[0] == value: # found tie data
            for index in item[1]: # insert all indices of value in price
              rankedIndices.append(index)
      else: # price is unique (no ties), add to ranking
        index = prices.index(value)
        rankedIndices.append(index)
  return rankedIndices

def greedy(data):
  currentPicks = [0]*len(data[0]) # array for storing our shopping list
  rankedChoices = getRanking(data[0], data[1]) # rank prices from high to low, solve ties, and store their index in order

  for i in rankedChoices:
    if validWeight(data[0], updateList(currentPicks, i), data[2]): # check if next item can fit
      currentPicks = updateList(currentPicks, i) # add item to cart

  return getPrice(data[1], currentPicks), currentPicks


In [49]:
solutions_greedy = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = greedy(row)
    solutions_greedy.append(1 if target == solution else 0)


In [50]:
print("Greedy Accuracy is", np.mean(solutions_greedy))

Greedy Accuracy is 0.92063974086446


**Your Analysis:**

Unsurprisingly, the greedy search did not return all optimal solutions. However it was surprising how accurate the algorithm was given that it is prone to finding and "getting stuck in" local maximums.

------------------------------------------------------------------------------------------------

**6. Simulated Annealing**

Next we will implement the simulated annealing algorithm. In this process, we make a random change to our selection and accept the change if it results in a desirable result. A seemingly less desirable result may also be accepted at random and with respect to "temperature".

The temperature starts with a high value and decreases with each iteration. A higher temperature increases the likelihood that a seemingly undesirable solution may be selected. This means that it will first explore many "poor" solutions. This is in an effort to escape from any local minima. However, the algorithm works, it will increasingly favor "better" solutions to (hopefully) reach a global maximum.

In our implementation, "energy" will be represented as price, and larger values will be preferred over smaller values. This differs from the classic case where the goal is to reach the global minima.


In [63]:
import random
import math

# Note: This solution uses the following functions as defined in the generate and test section:
# getPrice(prices, picks)
# validWeight(weights, picks, capacity)s]

# acceptence function
def accept(temperature, priceDifference):
  if priceDifference>0: # positive values are always accepted as they indicate increase in total price
    return True
  else:
    r = random.randint(0,1)
    n = pow(math.e, priceDifference/temperature) # allow for variation in acceptence
    if r < n:
      return True # allow worse selection
    else:
      return False

# given a list of binary values, return a list with one bit flipped
def flipRandomBit(array):
  index = random.choice(range(len(array))) # choose an index to alter
  out = array.copy()
  out[index] = (out[index]+1)%2 # flip bit
  return out

# return a new selection by flipping a random bit
# if capacity and weights are provided, only produce valid solutions
def modifyPicks(picks, weights=[], capacity=-1, acceptAny=False):

  newPicks = flipRandomBit(picks)

  # case 1: produce any new picks
  if capacity < 0 or acceptAny:
    return newPicks

  # case 2: produce new valid new picks
  while not validWeight(newPicks, weights, capacity):
    newPicks = flipRandomBit(picks)
  return newPicks

def simulated_annealing(data, N, initial_temperature, cooling_rate):
  temperature = initial_temperature
  currentPicks = greedy(data)[1] # start with a heuristic solution
  currentPrice = getPrice(data[1], currentPicks)

  newPicks = []
  newPrice = 0
  temperatureDifference = 0

  while (N > 0):
    newPicks = modifyPicks(currentPicks, data[0], data[2]) # get a new pick list
    newPrice = getPrice(data[1], newPicks)
    priceDifference = newPrice - currentPrice
    if accept(temperature, priceDifference): # acceptor function
      currentPicks = newPicks
      currentPrice = newPrice
    N = N - 1
    temperature = temperature * cooling_rate # reduce temperature by cooling rate

  return getPrice(data[1], currentPicks), currentPicks


In [52]:
solutions_sa = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = simulated_annealing(row, N = 1, initial_temperature=.10, cooling_rate=0.99)
    solutions_sa.append(1 if target == solution else 0)


In [53]:
print("Simulated Annealing Accuracy is", np.mean(solutions_sa))

Simulated Annealing Accuracy is 0.4506529000911023


**Your Analysis:**

Ti = .1, N = 10, cooling_rate = .95, accuracy = 0.20498026116003645

Ti = .1, N = 100, cooling_rate = .95, accuracy = 0.6160542565036947

Ti = .1, N = 1000, cooling_rate = .95, accuracy = 0.7374228160745014

Ti = .1, N = 1000, cooling_rate = .99, accuracy = 0.8951310861423221

Ti = .1, N = 5000, cooling_rate = .99, accuracy = 0.8901710699463509, time: 11min

Ti = .1, N= 5000, cooling_rate = .95, accuracy = 0.743597530114384, time: 11min

Ti = .5, N = 1000, cooling_rate = .99, accuracy = 0.8955359854236259, time 2min

Ti = .5, N = 1000, cooling_rate = .95, accuracy = 0.7351958700273307, time: 3min

Ti = .25, N = 1000, cooling_rate = .99, accuracy = 0.8887539224617876,time = 2min

Ti = .10, N = 1000, cooling_rate = .999, accuracy = 0.1827108006883288,time = 2min

Ti = .10, N = 1000, cooling_rate = .99, accurace = 0.895333535782974, time 2min

------------------------------------------------------------------------------------------------

**7. Genetic Algorithm**

Give a description... Also, make sure that you put comments using your own words in the code to show that you understand the code that you are submitting.

In [281]:
import random
import bisect

# Determine if anything else can fit without overflow
def underWeight(weights, picks, capacity):
  for i in range(len(picks)):
    if picks[i] == 0:
      if validWeight(weights, updateList(picks, i), capacity):
        return True
  return False

def calculate_fitness(picks, prices, weights, capacity):
  fitnessWeights = [2,1,1,1,1] # not finished
  includeFactor = [True, True, True, True] # implement me if runtime is too long
  fitness = 100
  bagValue = getPrice(prices, picks)
  validWeightBool = validWeight(weights, picks, capacity)
  underWeightBool = underWeight(weights, picks, capacity)

  # clean this  up. ############################################################

  # overweight? not fit.
  if not validWeightBool:
    fitness = fitness + fitnessWeights[0]*100
    fitness = fitness + 10*(getPrice(weights, picks)/capacity)
  # can another item fit? not fit.
  elif underWeightBool:
    fitness = fitness + fitnessWeights[1]*100
    # encourage lower overall weights
    # or encourage higher? i dont know whats better.
    fitness = fitness + 10*(getPrice(weights, picks)/capacity)

  # is price less than greedy? not fit.
  greedyPrice = greedy([weights, prices, capacity])[0]
  if greedyPrice == bagValue and validWeightBool:
    #fitness = fitness*.99
    pass
  elif bagValue == 0:
    fitness = fitness + 200
  # worse solutions are proportionally punished
  elif greedyPrice > bagValue:
    fitness = fitness * (greedyPrice/bagValue)
    if underWeightBool:
      fitness = fitness + 100
  # did we do better than greedy? lets encourage this proportionally to how much better we did
  elif greedyPrice < bagValue and validWeightBool:
    fitness = 100 * (greedyPrice/bagValue) # implement weight function
  return fitness

# Function to test how different weights affect fitness function
def fitnessAlgoAnalyzer():
  picksTest = binary_permutation(3)
  pricesTest = [10, 20, 30]
  weightsTest = [5, 10, 10]
  capacityTest = 20
  report = []
  report.append(["Gen and test (price, picks)",gen_and_test([weightsTest,pricesTest,capacityTest])])
  report.append(["Greedy (price, picks):", greedy([weightsTest,pricesTest,capacityTest])])

  for testPick in picksTest:
    line = []
    line.append(["Pick:",testPick])
    line.append(["Price:",getPrice(pricesTest,testPick)])
    line.append(["Overweight:", not validWeight(weightsTest, testPick, capacityTest)])
    line.append(["Underweight:", underWeight(weightsTest, testPick, capacityTest)])
    line.append(["Fitness:", calculate_fitness(testPick,pricesTest,weightsTest,capacityTest)])
    report.append(line)
  for line in report:
    print(line)
#fitnessAlgoAnalyzer()



# you can use one point crossover .point can be fixed or variable
# can also explore other crossovers
# explore different rates for crossover and mutation.
# analysis: explore 2 other rates (one lower and one higher) for each, whats the impact?

def crossover(parent1, parent2, cross_rate):
  if random.random() < cross_rate:
    n = random.randint(1, len(parent1) - 1)
    child1 = parent1[:n] + parent2[n:]
    child2 = parent2[:n] + parent1[n:]
    return child1, child2
  else:
    return parent1, parent2

def applyCrossover(population, crossRate):
  indices = []
  newPopulation = []

  #for i in range(len(population)):
  #  indices.append(i)

  for i in range(len(population)//2):
    parent1 = population.pop(random.randint(0, len(population)-1))
    parent2 = population.pop(random.randint(0, len(population)-1))
    child1, child2 = crossover(parent1, parent2, crossRate)
    newPopulation.append(child1)
    newPopulation.append(child2)

  if len(population) > 0:
    newPopulation.append(population[0])

  return newPopulation


def mutation(child, mut_rate):
  if random.random() < mut_rate:
    child = flipRandomBit(child)
  return child

def applyMutation(population, mutationRate):
  newPopulation = []

  for individual in population:
    newPopulation.append(mutation(individual, mutationRate))

  return newPopulation

# return the liklihood of selecting the nth best individual given probability p
def getSelectionProbability(n, p):
  return p*(pow((1-p),n))

# run tournament cycle
def runTournament(data, participants, probability):

  if len(participants)==0:
    return None

  rankedParticipants = []
  for individual in participants:
    fitness = calculate_fitness(individual, data[1], data[0], data[2])
    bisect.insort(rankedParticipants, [fitness, individual], key=lambda x: x[0])

  # deterministic approach
  if probability == 1:
    return rankedParticipants[0][1]

  # non-deterministic, get weights
  selectionWeights = []
  for i in range(len(rankedParticipants)):
    selectionWeights.append(getSelectionProbability(i, probability))
  return random.choices(rankedParticipants, weights=selectionWeights, cum_weights=None, k=1)[0][1]

def randomSample(population, size):
  used = []
  sample = []
  index = 0
  if len(population)<size:
    return population
  while len(sample) != size:
    index = random.choice(range(size))
    if index not in used:
      used.append(index)
      sample.append(population[index])
  return sample

def getSelectionByTournament(data, population, populationSize, tournamentSize, probability, replacement=False):
  winner = []
  workingPopulation = population.copy()
  newPopulation = []
#or len(workingPopulation) ==0
  while len(newPopulation) < populationSize :
    participants = randomSample(workingPopulation, tournamentSize)
    winner = runTournament(data, participants, probability)
    if winner == None:
      break
    newPopulation.append(winner)
    if not replacement:
      workingPopulation.remove(winner)

  return newPopulation


#print(randomSample(testPop, 10))
#print("best sol:",gen_and_test(testData))
#selection = getSelectionByTournament(testData,testPop,3,10,.8)
#print("selection len:",len(selection),"pop len:",len(testPop))
#print("after tornament:", selection)
#print(runTournament(testData, testPop, .8))
#Generate a population X(t) of M individuals

#repeat
	#Select a subset according to fitness
	#Perform variations on these individuals to generate new individuals in X(t+1)
#Untill stopping condition
def genetic_algorithm(data, population_size, num_generations, mut_rate, cross_rate, tournament_size):
  population = randomSample(binary_permutation(5), population_size)

  for i in range(num_generations):
    #print("Pop length before torny:",len(population))
    population = getSelectionByTournament(data, population, population_size, tournament_size, .8, False)
    #print("Pop length before crossover:",len(population))
    population = applyCrossover(population, cross_rate)
    #print("Pop length before mutation:",len(population))
    population = applyMutation(population, mut_rate)

  best_solution = getSelectionByTournament(data, population, population_size, tournament_size, 1)[0]
  best_solution_price = getPrice(data[1], best_solution)

  #print(getSelectionByTournament(data, population, population_size, tournament_size, 1))
  #print("ga algo:",best_solution)
  #print("gen and test:", gen_and_test(data))


  return best_solution_price, best_solution

#testData = [[5,10,20,10,50],[10,5,20,10,100],30]
#genetic_algorithm(testData,population_size=50,num_generations=50,mut_rate=.1,cross_rate=.7,tournament_size=5)

In [ ]:
solutions_ga = []
for _, row in dataset.iterrows():
    target = row['Best price']
    solution, indexes = genetic_algorithm(row, population_size = 50, num_generations = 50, mut_rate = 0.1, cross_rate = 0.7, tournament_size = 5)
    solutions_ga.append(1 if target == solution else 0)


In [1]:
print("Genetic Algorithm Accuracy is", np.mean(solutions_ga))

NameError: ignored

**Your Analysis:**

------------------------------------------------------------------------------------------------

**8. Comparative Study**

description  +  tables/figures

--------------------------------------------------------------------------


**9. Conclusion**

Comment on the empirical study, its results, and give ideas for future work.

--------------------------------------------------------------------------


# **10. References**

Make sure you provide references to ALL sources used (articles, code, algorithms).

gen and test: https://www.geeksforgeeks.org/python-using-2d-arrays-lists-the-right-way/

greedy: https://docs.python.org/3/howto/sorting.html
        https://stackoverflow.com/questions/12897374/get-unique-values-from-a-list-in-python

SA: https://www.baeldung.com/cs/simulated-annealing

GA: https://www.baeldung.com/cs/genetic-algorithms-crossover-probability-and-mutation-probability
https://ai.stackexchange.com/questions/9105/how-to-create-a-good-fitness-function
https://stackoverflow.com/questions/8024571/insert-an-item-into-sorted-list-in-python


**Hint:** To share a link to your colab notebook, click on "share" on the top right. Then, under *General access* , change *Restricted* to "Anyone with the link".